In [1]:
import torch
import pathlib
import cv2
import PIL
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
%matplotlib inline

from passion.segmentation import prediction

In [2]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

# Load testing models

In [3]:
models_folder_path = pathlib.Path('../../workflow/output/model')
superstructures_models_folder_path = models_folder_path / 'superst-segmentation'
rooftop_models_folder_path = models_folder_path / 'rooftop-segmentation'

# Sections models
model_aug_15e = torch.load(str(superstructures_models_folder_path / 'superstructures-rid-15e.pth'), map_location=torch.device(device))
model_aug_20e = torch.load(str(superstructures_models_folder_path / 'superstructures-rid-20e.pth'), map_location=torch.device(device))
model_raw_20e = torch.load(str(superstructures_models_folder_path / 'superstructures-rid-raw-20e.pth'), map_location=torch.device(device))

# Rooftop model
model_inria = torch.load(str(rooftop_models_folder_path / 'inria-aug-12e.pth'), map_location=torch.device(device))

# Testing folder paths

In [4]:
rid_test_folder = pathlib.Path('/storage/internal/home/r-pueblas/projects/rooftop-segmentation-datasets/data/RID/output/masks_superstructures_reviewed/test')
rid_test_folder_image = rid_test_folder / 'image'
rid_test_folder_label = rid_test_folder / 'label'

# Metrics

In [5]:
def intersect_and_union(pred_label, label, num_classes, ignore_index):
    mask = (label != ignore_index)
    pred_label = pred_label[mask]
    label = label[mask]

    intersect = pred_label[pred_label == label]
    area_intersect, _ = np.histogram(
        intersect, bins=np.arange(num_classes + 1))
    area_pred_label, _ = np.histogram(
        pred_label, bins=np.arange(num_classes + 1))
    area_label, _ = np.histogram(label, bins=np.arange(num_classes + 1))
    area_union = area_pred_label + area_label - area_intersect

    return area_intersect, area_union, area_pred_label, area_label

# Helper functions

In [6]:
def test_model_superstructures(model, test_folder, num_classes, background_class, ignore_index, output=True):
    start = time.time()
    
    test_folder_image = test_folder / 'image'
    test_folder_label = test_folder / 'label'

    total_area_intersect = np.zeros((num_classes, ), dtype=float)
    total_area_union = np.zeros((num_classes, ), dtype=float)
    total_area_pred_label = np.zeros((num_classes, ), dtype=float)
    total_area_label = np.zeros((num_classes, ), dtype=float)

    for i, filename in enumerate(test_folder_image.glob('*.png')):
        image = cv2.imread(str(filename))
        label = cv2.imread(str(test_folder_label / filename.name))
        label = label[:,:,0]


        pred = prediction.segment_img(image, model, tile_size=512, stride=512, background_class=background_class)

        area_intersect, area_union, area_pred_label, area_label = \
                            intersect_and_union(pred, label, num_classes, ignore_index)
        total_area_intersect += area_intersect
        total_area_union += area_union
        total_area_pred_label += area_pred_label
        total_area_label += area_label

        if i<5 and output:
            f, axarr = plt.subplots(1,3, figsize=(10, 10))
            axarr[0].imshow(image)
            axarr[1].imshow(label)
            axarr[2].imshow(pred)
            if i==0:
                axarr[0].title.set_text('Image')
                axarr[1].title.set_text('Ground truth')
                axarr[2].title.set_text('model_mix_aug')

            plt.subplots_adjust(wspace=0.1, hspace=0.1)
            for ax in axarr:
                ax.axis('off')
            plt.show()

        iou = total_area_intersect / total_area_union

        if output: print(f'Processed image {filename.stem}, partial mean iou: {np.mean(iou[:-1][~np.isnan(iou[:-1])])}, with background class: {np.mean(iou[~np.isnan(iou)])}')

    all_acc = total_area_intersect.sum() / total_area_label.sum()
    acc = total_area_intersect / total_area_label
    iou = total_area_intersect / total_area_union

    end = time.time()
    elapsed_time = (end - start)

    if output:
        print('\n')
        print(f'Elapsed time: {elapsed_time} seconds')
        print(f'Final IoU per class: {iou}')
        print(f'Final mean IoU with background class: {np.mean(iou[~np.isnan(iou)])}')
        print(f'Final mean IoU without background class: {np.mean(iou[:-1][~np.isnan(iou[:-1])])}')
    return iou, np.mean(iou[~np.isnan(iou)]), elapsed_time

def test_model_twolook(rooftop_model,
                       superstructures_model,
                       test_folder,
                       num_classes,
                       background_class_1,
                       background_class_2,
                       ignore_index,
                       output=True
                      ):
    start = time.time()
    
    test_folder_image = test_folder / 'image'
    test_folder_label = test_folder / 'label'

    total_area_intersect = np.zeros((num_classes, ), dtype=float)
    total_area_union = np.zeros((num_classes, ), dtype=float)
    total_area_pred_label = np.zeros((num_classes, ), dtype=float)
    total_area_label = np.zeros((num_classes, ), dtype=float)

    for i, filename in enumerate(test_folder_image.glob('*.png')):
        image = cv2.imread(str(filename))
        label = cv2.imread(str(test_folder_label / filename.name))
        label = label[:,:,0]
    
        roofs = prediction.segment_img(image, rooftop_model, tile_size=512, stride=512, background_class=background_class_1)
        filtered = np.ones(image.shape).astype(np.uint8)
        idx=(roofs!=0)
        filtered[idx] = image[idx]

        pred = prediction.segment_img(image, superstructures_model, tile_size=512, stride=512, background_class=background_class_2)
        pred = (pred * idx.astype('uint8'))
        area_intersect, area_union, area_pred_label, area_label = \
                            intersect_and_union(pred, label, num_classes, ignore_index)
        total_area_intersect += area_intersect
        total_area_union += area_union
        total_area_pred_label += area_pred_label
        total_area_label += area_label

        if i<5 and output:
            f, axarr = plt.subplots(1,3, figsize=(10, 10))
            axarr[0].imshow(image)
            axarr[1].imshow(label)
            axarr[2].imshow(pred)
            if i==0:
                axarr[0].title.set_text('Image')
                axarr[1].title.set_text('Ground truth')
                axarr[2].title.set_text('model_mix_aug')

            plt.subplots_adjust(wspace=0.1, hspace=0.1)
            for ax in axarr:
                ax.axis('off')
            plt.show()

        iou = total_area_intersect / total_area_union

        if output: print(f'Processed image {filename.stem}, partial mean iou: {np.mean(iou[:-1][~np.isnan(iou[:-1])])}, with background class: {np.mean(iou[~np.isnan(iou)])}')

    all_acc = total_area_intersect.sum() / total_area_label.sum()
    acc = total_area_intersect / total_area_label
    iou = total_area_intersect / total_area_union

    end = time.time()
    elapsed_time = (end - start)

    if output:
        print('\n')
        print(f'Elapsed time: {elapsed_time} seconds')
        print(f'Final IoU per class: {iou}')
        print(f'Final mean IoU with background class: {np.mean(iou[~np.isnan(iou)])}')
        print(f'Final mean IoU without background class: {np.mean(iou[:-1][~np.isnan(iou[:-1])])}')

    return iou, np.mean(iou[~np.isnan(iou)]), elapsed_time

# Results

In [7]:
num_classes = 9
background_class = 0
ignore_index = -1

In [8]:
print(f'Testing model aug 15e...')

iou, mean_iou, elapsed_time = test_model_superstructures(model_aug_15e,
                                                              rid_test_folder,
                                                              num_classes,
                                                              background_class,
                                                              ignore_index,
                                                              output=False)

print(f'Mean IoU: {mean_iou}')
print(f'IoU per class: {iou}')
print(f'Elapsed time: {elapsed_time} seconds.')

Testing model aug 15e...


/tmp/ipykernel_1089492/1480525181.py:42: RuntimeWarning: invalid value encountered in true_divide
  iou = total_area_intersect / total_area_union


Mean IoU: 0.46320181844823033
IoU per class: [0.80885863 0.55937964 0.43855717 0.11804009 0.35329789 0.4580988
 0.28377423 0.16848972 0.9803202 ]
Elapsed time: 99.74971318244934 seconds.


In [9]:
print(f'Testing model aug 20e...')

iou, mean_iou, elapsed_time = test_model_superstructures(model_aug_20e,
                                                              rid_test_folder,
                                                              num_classes,
                                                              background_class,
                                                              ignore_index,
                                                              output=False)

print(f'Mean IoU: {mean_iou}')
print(f'IoU per class: {iou}')
print(f'Elapsed time: {elapsed_time} seconds.')

Testing model aug 20e...


/tmp/ipykernel_1089492/1480525181.py:42: RuntimeWarning: invalid value encountered in true_divide
  iou = total_area_intersect / total_area_union


Mean IoU: 0.4927006571185375
IoU per class: [0.81640351 0.59532834 0.47644888 0.17905137 0.41627993 0.48439763
 0.2902169  0.1948382  0.98134116]
Elapsed time: 101.74772477149963 seconds.


In [10]:
print(f'Testing model raw 20e...')

iou, mean_iou, elapsed_time = test_model_superstructures(model_raw_20e,
                                                              rid_test_folder,
                                                              num_classes,
                                                              background_class,
                                                              ignore_index,
                                                              output=False)

print(f'Mean IoU: {mean_iou}')
print(f'IoU per class: {iou}')
print(f'Elapsed time: {elapsed_time} seconds.')

Testing model raw 20e...


/tmp/ipykernel_1089492/1480525181.py:42: RuntimeWarning: invalid value encountered in true_divide
  iou = total_area_intersect / total_area_union


Mean IoU: 0.3672405402297685
IoU per class: [7.32984870e-01 4.48470338e-01 3.47538933e-01 4.19064648e-05
 1.62660642e-01 3.74906068e-01 1.73192733e-01 8.90930975e-02
 9.76276274e-01]
Elapsed time: 101.1682550907135 seconds.


# Two-look

In [11]:
background_class_1 = 0
background_class_2 = 0

print(f'Testing two look models...')
iou, mean_iou, elapsed_time = test_model_twolook(model_inria,
                                                 model_aug_20e,
                                                 rid_test_folder,
                                                 num_classes,
                                                 background_class_1,
                                                 background_class_2,
                                                 ignore_index,
                                                 output=False)

print(f'Mean IoU: {mean_iou}')
print(f'IoU per class: {iou}')
print(f'Elapsed time: {elapsed_time} seconds.')

Testing two look models...


/tmp/ipykernel_1089492/1480525181.py:114: RuntimeWarning: invalid value encountered in true_divide
  iou = total_area_intersect / total_area_union


Mean IoU: 0.253602244209725
IoU per class: [0.01269244 0.55328723 0.43602122 0.16382758 0.35548558 0.32489318
 0.06409266 0.15899628 0.21312404]
Elapsed time: 203.6452877521515 seconds.
